In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import pickle
import json
import torch

from PIL import Image
from matplotlib import patches
from pathlib import Path
from tqdm.autonotebook import tqdm
from typing import Dict, List, Tuple

from extract_features import get_all_annotations

In [9]:
dataset_file = 'annotations/midog_2022_test.csv'
img_dir = '/data/patho/MIDOG2/finalTest/'
figure_dir = Path('figures/')
result_dir = Path('results/')
feature_dir = Path('/data/jonas/midog/features')

In [30]:
import utils.constants as constants

# load data 
dataset = pd.read_csv(dataset_file)

# filter test samples 
test_dataset = dataset.query('split == "test"')

# create test codes
test_codes = {k: v for k, v in enumerate(test_dataset['tumortype'].unique())}


# get test samples and labels
test_samples = get_all_annotations(
    dataset=test_dataset, 
    img_dir_path=img_dir, 
    domain_col='tumortype'
    )

# testset labels
test_annos = torch.tensor([v for l in test_samples.values() for v in l['labels']])
if 'midog' in dataset_file.lower():
    test_annos -= 1


In [22]:
model_name = 'yolov7_d6_ALL_0'

print('Loading features and targets ...', end=' ')
if not feature_dir.joinpath('features_' + model_name + '.pkl').exists():
    raise FileNotFoundError(f'Features for model {model_name} not found.')
else:
    features = pickle.load(open(feature_dir.joinpath('features_' + model_name + '.pkl'), 'rb'))

if not feature_dir.joinpath('domains_' + model_name + '.pkl').exists():
    raise FileNotFoundError(f'Domains for model {model_name} not found.')
else:
    domains = pickle.load(open(feature_dir.joinpath('domains_' + model_name + '.pkl'), 'rb'))
print('Done.')


Loading features and targets ... Done.


In [23]:
for name, values in features.items():
    print(name, values.shape)

model.14 torch.Size([10301, 192])
model.27 torch.Size([10301, 384])
model.40 torch.Size([10301, 768])
model.53 torch.Size([10301, 1152])
model.66 torch.Size([10301, 1536])
model.83 torch.Size([10301, 576])
model.99 torch.Size([10301, 384])
model.115 torch.Size([10301, 192])
model.129 torch.Size([10301, 384])
model.143 torch.Size([10301, 576])
model.157 torch.Size([10301, 768])
model.158 torch.Size([10301, 384])
model.159 torch.Size([10301, 768])
model.160 torch.Size([10301, 1152])
model.161 torch.Size([10301, 1536])
model.166.m.0 torch.Size([10301, 18])
model.166.m.1 torch.Size([10301, 18])
model.166.m.2 torch.Size([10301, 18])
model.166.m.3 torch.Size([10301, 18])


In [26]:
import umap 
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pandas as pd

In [36]:
reducer = umap.UMAP()

scaled_features = StandardScaler().fit_transform(features['model.166.m.0'])
mapper = reducer.fit(scaled_features)

In [39]:
import umap.plot

umap.plot.points(mapper)

/home/ammeling/projects/yolov7/env/lib/python3.10/site-packages/umap/plot.py:449: UserWarning: *c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
  ax.scatter(points[:, 0], points[:, 1], s=point_size, c=color)


<Axes: >

In [34]:
embedding


array([[    -5.3993,      8.6397],
       [    0.77022,      11.672],
       [     7.5191,      9.1877],
       ...,
       [     6.5742,      10.226],
       [     2.0526,      8.5728],
       [    0.49043,      11.849]], dtype=float32)